In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("C:/Users/david/Downloads/malicious_phish.csv") 
df

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement
...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing
651188,www.gamespot.com/xbox360/action/deadspace/,phishing
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing


In [3]:
df["type"].unique()

array(['phishing', 'benign', 'defacement', 'malware'], dtype=object)

In [8]:
df.isnull().sum()

url     0
type    0
dtype: int64

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['url'])  # Matriz TF-IDF
y = df['type']  # Etiquetas

In [8]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

print("Antes del submuestreo:", Counter(y))

# Definir el tamaño de las clases balanceadas (por defecto iguala a la clase minoritaria)
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

print("Después del submuestreo:", Counter(y_resampled))


Antes del submuestreo: Counter({'benign': 428103, 'defacement': 96457, 'phishing': 94111, 'malware': 32520})
Después del submuestreo: Counter({'benign': 32520, 'defacement': 32520, 'malware': 32520, 'phishing': 32520})


In [9]:
from sklearn.model_selection import train_test_split

def split_train_val_test(X, y, random_state=42):
    """
    Divide X e y en train (60%), val (20%) y test (20%), conservando la proporción de clases.
    
    Parámetros:
    - X: Matriz de características (sparse o dense)
    - y: Vector de etiquetas
    - random_state: Semilla para reproducibilidad

    Retorna:
    - X_train, X_val, X_test, y_train, y_val, y_test
    """
    
    # Primera división: 60% train, 40% temp (val+test)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.4, random_state=random_state, stratify=y
    )
    
    # Segunda división: 20% val, 20% test
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=random_state, stratify=y_temp
    )
    
    return X_train, X_val, X_test, y_train, y_val, y_test


In [10]:
X_train, X_val, X_test, y_train, y_val, y_test = split_train_val_test(X_resampled, y_resampled)


In [11]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

# Definir el modelo SVM con kernel lineal
clf = LinearSVC(random_state=42)

# Entrenar con el set de entrenamiento
clf.fit(X_train, y_train)

# Validación
y_val_pred = clf.predict(X_val)
print("Resultados en Validación:")
print(classification_report(y_val, y_val_pred))

# Test final
y_test_pred = clf.predict(X_test)
print("Resultados en Test:")
print(classification_report(y_test, y_test_pred))


Resultados en Validación:
              precision    recall  f1-score   support

      benign       0.84      0.94      0.88      6504
  defacement       0.98      1.00      0.99      6504
     malware       0.99      0.97      0.98      6504
    phishing       0.91      0.81      0.86      6504

    accuracy                           0.93     26016
   macro avg       0.93      0.93      0.93     26016
weighted avg       0.93      0.93      0.93     26016

Resultados en Test:
              precision    recall  f1-score   support

      benign       0.83      0.94      0.88      6504
  defacement       0.98      1.00      0.99      6504
     malware       0.99      0.97      0.98      6504
    phishing       0.91      0.81      0.86      6504

    accuracy                           0.93     26016
   macro avg       0.93      0.93      0.93     26016
weighted avg       0.93      0.93      0.93     26016



In [ ]:
x = 2
x